<a href="https://colab.research.google.com/github/moey920/Object-Detection/blob/master/detectron_batch_process.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
# 기본 설정
# detectron2 logger 설정
import detectron2
from detectron2.utils.logger import setup_logger

setup_logger()

# 자주 사용하는 라이브러리 임폴트
import numpy as np
import cv2
import random
import os
import fnmatch

# from google.colab.patches import cv2_imshow
# import json

# import itertools
# import tkinter
# import matplotlib
# import matplotlib.patches as patches
# import matplotlib.pyplot as plt
# from matplotlib.transforms import Bbox

'''
print("debug ----- matplotlib.get_backend()", matplotlib.get_backend())
print("debug ----- matplotlib.is_interactive", matplotlib.is_interactive())
matplotlib.pyplot.ion()
print("debug ----- matplotlib.is_interactive", matplotlib.is_interactive())
'''
# matplotlib.use('TkAgg')
# matplotlib.use('Qt5Agg')
# matplotlib.use('TKAgg',warn=False, force=True)
# print(matplotlib.get_backend())


from multiprocessing import freeze_support

# 자주 사용하는 detectron2 유틸 임폴트
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
# from detectron2.utils.visualizer import Visualizer
from extVisualizer import ExtVisualizer
# from detectron2.data import MetadataCatalog
# from detectron2.structures import BoxMode
# from detectron2.data import DatasetCatalog, MetadataCatalog
# from detectron2.data.datasets import register_coco_instances
# from detectron2.data.detection_utils import *
from detectron2.utils.visualizer import ColorMode, GenericMask
# from detectron2.utils.visualizer import VisImage
# from detectron2.structures import BitMasks, Boxes, BoxMode, Keypoints, PolygonMasks, RotatedBoxes


from scipy.spatial import ConvexHull


# from sklearn.cluster import KMeans
def get_outer_vertex(convex, w, h):
    top = []
    bot = []
    xmin, ymin = np.amin(convex, axis=0)
    xmax, ymax = np.amax(convex, axis=0)
    xthres = w * 0.02
    ythres = h * 0.02

    pts = convex.tolist()
    # print("debug ---- count of points", len(pts))
    # print("debug ---- thresholds", xthres, ythres)

    # 상단 가로줄 획득. ymin 에 근사한 것들에서 구하기
    for p in sorted(pts, key=lambda pt: pt[1]):
        if abs(p[1] - ymin) < ythres:
            top.append(p)
            pts.remove(p)
            ymin = max(ymin, p[1])
        else:
            break

    # print("debug ---- count of top points", len(top))
    # print("debug ---- count of remain points", len(pts))

    # top band를 좌우로 분할
    top = sorted(top)
    xmin, _ = np.amin(np.asarray(top), axis=0)
    xmax, _ = np.amax(np.asarray(top), axis=0)
    tbl = [p for p in top if p[0] < int((xmin + xmax) / 2)]
    tbr = [p for p in top if p[0] > int((xmin + xmax) / 2)]
    tlxmin, tlymin = np.amin(np.asarray(tbl), axis=0)
    tlxmax, tlymax = np.amax(np.asarray(tbl), axis=0)
    # top left 점 획득
    tl = [tlxmin, tlymin]
    trxmin, trymin = np.amin(np.asarray(tbr), axis=0)
    trxmax, trymax = np.amax(np.asarray(tbr), axis=0)
    # top right 점 획득
    tr = [trxmax, trymin]

    # 하단 가로줄 획득. ymax에 기준
    for p in sorted(pts, key=lambda pt: pt[1], reverse=True):
        if abs(p[1] - ymax) < ythres:
            bot.append(p)
            pts.remove(p)
            ymax = min(ymax, p[1])
        else:
            break
    # print("debug ---- count of bottom points", len(bot))
    # print("debug ---- count of remain points", len(pts))

    # bottom band를 좌우로 분할
    bot = sorted(bot)
    xmin, _ = np.amin(np.asarray(bot), axis=0)
    xmax, _ = np.amax(np.asarray(bot), axis=0)
    bbl = [p for p in bot if p[0] < int((xmin + xmax) / 2)]
    bbr = [p for p in bot if p[0] > int((xmin + xmax) / 2)]
    blxmin, blymin = np.amin(np.asarray(bbl), axis=0)
    blxmax, blymax = np.amax(np.asarray(bbl), axis=0)
    # bottom left 점 획득
    bl = [blxmin, blymax]
    brxmin, brymin = np.amin(np.asarray(bbr), axis=0)
    brxmax, brymax = np.amax(np.asarray(bbr), axis=0)
    # bottom right 점 획득
    br = [brxmax, brymax]

    ret = {"tl": tl, "tr": tr, "br": br, "bl": bl}
    print("debug ----- outer vertices", ret)

    return ret


'''
def full_extent(ax, pad=0.0):
    """Get the full extent of an axes, including axes labels, tick labels, and
    titles."""
    # For text objects, we need to draw the figure first, otherwise the extents
    # are undefined.
    ax.figure.canvas.draw()
    # items = ax.get_xticklabels() + ax.get_yticklabels()
    # items += [ax, ax.title]
    items = [ax]
    bbox = Bbox.union([item.get_window_extent() for item in items])

    return bbox.expanded(1.0 + pad, 1.0 + pad)
'''

'''
def mask_outside_polygon(poly_verts, ax=None):
        """
        Plots a mask on the specified axis ("ax", defaults to plt.gca()) such that
        all areas outside of the polygon specified by "poly_verts" are masked.

        "poly_verts" must be a list of tuples of the verticies in the polygon in
        counter-clockwise order.

        Returns the matplotlib.patches.PathPatch instance plotted on the figure.
        """
        import matplotlib.patches as mpatches
        import matplotlib.path as mpath

        if ax is None:
            ax = plt.gca()

        # Get current plot limits
        xlim = ax.get_xlim()
        ylim = ax.get_ylim()

        # Verticies of the plot boundaries in clockwise order
        bound_verts = [(xlim[0], ylim[0]), (xlim[0], ylim[1]),
                       (xlim[1], ylim[1]), (xlim[1], ylim[0]),
                       (xlim[0], ylim[0])]

        # convex = ConvexHull(poly_verts)

        # 변환
        # "poly_verts" must be a list of tuples of the verticies in the polygon in counter-clockwise order.
        dist2 = lambda a, b: (a[0] - b[0]) * (a[0] - b[0]) + (a[1] - b[1]) * (a[1] - b[1])
        # z = list(zip(x, y))  # get the list of coordinate pairs
        # z = poly_verts[convex.vertices].tolist()
        z = poly_verts.tolist()
        z.sort()  # sort by x coordinate

        cw = z[0:1]  # first point in clockwise direction
        ccw = z[1:2]  # first point in counter clockwise direction
        # reverse the above assignment depending on how first 2 points relate
        if z[1][1] > z[0][1]:
            cw = z[1:2]
            ccw = z[0:1]

        for p in z[2:]:
            # append to the list to which the next point is closest
            if dist2(cw[-1], p) < dist2(ccw[-1], p):
                cw.append(p)
            else:
                ccw.append(p)

        cw.reverse()
        result = cw + ccw
        poly_verts = result
        poly_verts.append(poly_verts[0])

        # A series of codes (1 and 2) to tell matplotlib whether to draw a line or
        # move the "pen" (So that there's no connecting line)sk
        bound_codes = [mpath.Path.MOVETO] + (len(bound_verts) - 1) * [mpath.Path.LINETO]
        poly_codes = [mpath.Path.MOVETO] + (len(poly_verts) - 1) * [mpath.Path.LINETO]

        # Plot the masking patch
        path = mpath.Path(bound_verts + poly_verts, bound_codes + poly_codes)
        patch = mpatches.PathPatch(path, facecolor='white', edgecolor='none')
        patch = ax.add_patch(patch)

        # Reset the plot limits to their original extents
        ax.set_xlim(xlim)
        ax.set_ylim(ylim)

        return patch
'''


def save_masked_image(opath, convex):
    try:
        # original image
        # -1 loads as-is so if it will be 3 or 4 channel as the original
        image = cv2.imread(opath, -1)
        # mask defaulting to black for 3-channel and transparent for 4-channel
        # (of course replace corners with yours)
        mask = np.zeros(image.shape, dtype=np.uint8)
        # mask = np.ones(image.shape, dtype=np.uint8)
        # roi_corners = np.array([[(10, 10), (300, 300), (10, 300)]], dtype=np.int32)
        roi_corners = np.array(convex.tolist(), dtype=np.int32)
        # fill the ROI so it doesn't get wiped out when the mask is applied
        channel_count = image.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
        # ignore_mask_color = (0,) * channel_count
        # cv2.fillPoly(mask, roi_corners, ignore_mask_color)
        # from Masterfool: use cv2.fillConvexPoly if you know it's convex
        cv2.fillConvexPoly(mask, roi_corners, ignore_mask_color)

        # apply the mask
        masked_image = cv2.bitwise_and(image, mask)

        # save the result
        savefilename = "{}_masked.png".format(os.path.splitext(opath)[0])
        cv2.imwrite(savefilename, masked_image)
    except cv2.error as e:
        savefilename = ""
        None
        print("Err:")
        print(e)

    return savefilename


def check_pts(vs):
    "상단/하단 가로 길이 격차가 너무 크거나 좌/우 세로 길이 격차가 너무 크지 않는지 판단"
    # i 가 0이면 x 좌표, 1이면 y 좌표
    a = abs(vs['tr'][0] - vs['tl'][0])
    b = abs(vs['br'][0] - vs['bl'][0])
    # 둘 중 큰 것의 반
    m = max(a, b) / 2
    if (a < m) or (b < m):
        return False

    a = abs(vs['br'][1] - vs['tr'][1])
    b = abs(vs['bl'][1] - vs['tl'][1])
    # 둘 중 큰 것의 반
    m = max(a, b) / 2
    if (a < m) or (b < m):
        return False

    return True


if __name__ == '__main__':
    freeze_support()

    # register_coco_instances("train", {}, "train/receipt_coco.json", "train/")
    # register_coco_instances("val", {}, "val/receipt_coco.json", "val/")

    # receipt_metadata_train = MetadataCatalog.get("train")
    # receipt_metadata_val = MetadataCatalog.get("val")

    cfg = get_cfg()
    cfg.merge_from_file("./detectron2_repo/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
    cfg.DATALOADER.NUM_WORKERS = 2
    cfg.SOLVER.IMS_PER_BATCH = 2
    cfg.SOLVER.BASE_LR = 0.00025
    cfg.SOLVER.MAX_ITER = 1000
    cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 1024
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1

    cfg.MODEL.WEIGHTS = os.path.join('output', "model_final.pth")
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7  # set the testing threshold for this model
    # cfg.DATASETS.TEST = ("val",)
    predictor = DefaultPredictor(cfg)

    mypath = "image"
    i = 0
    files = [os.path.join(mypath, f) for f in os.listdir(mypath) if os.path.isfile(os.path.join(mypath, f))]
    # for d in [dataset_dicts[3]]:
    # for d in random.sample(files, 1):
    for d in files:

        if fnmatch.fnmatch(d, '*_masked.png') or fnmatch.fnmatch(d, '*_transform.png'):
            continue

        # for d in random.sample(dataset_dicts, 4):
        filepath = d
        im = cv2.imread(filepath)

        print("debug ----- {} th image".format(i), filepath)

        outputs = predictor(im)
        v = ExtVisualizer(im[:, :, ::-1],
                          # metadata=receipt_metadata_val,
                          # scale=0.8,
                          scale=1.0,
                          instance_mode=ColorMode.SEGMENTATION
                          )

        # box, mask 관련 추출
        predictions = outputs["instances"].to("cpu")

        boxes = predictions.pred_boxes if predictions.has("pred_boxes") else None
        if predictions.has("pred_masks"):
            masks = np.asarray(predictions.pred_masks)
            masks = [GenericMask(x, v.output.height, v.output.width) for x in masks]
        else:
            masks = None

        if boxes is None or masks is None:
            print("warning ----- {} th. there is no boxes nor masks >> pass".format(i))
            continue

        num_instances = None
        if boxes is not None:
            boxes = v._convert_boxes(boxes)
            num_instances = len(boxes)
        if masks is not None:
            masks = v._convert_masks(masks)
            if num_instances:
                assert len(masks) == num_instances
            else:
                num_instances = len(masks)

        # Display in largest to smallest order to reduce occlusion.
        areas = None
        if boxes is not None:
            areas = np.prod(boxes[:, 2:] - boxes[:, :2], axis=1)
        elif masks is not None:
            areas = np.asarray([x.area() for x in masks])

        if areas is not None:
            sorted_idxs = np.argsort(-areas).tolist()
            # Re-order overlapped instances in descending order.
            boxes = boxes[sorted_idxs] if boxes is not None else None
            masks = [masks[idx] for idx in sorted_idxs] if masks is not None else None

        if masks is not None and len(masks) > 0:
            for segment in masks[0].polygons:
                seg = segment.reshape(-1, 2)
                # convex = hull.convex(tuple(map(tuple, seg)))
                convex = ConvexHull(seg)
                # print("debug ----- masks[0] segments convex hull vertices", convex.vertices)
        else:
            print("warning ----- {} th. there is no masks[0] >> pass".format(i))
            continue

        # v = v.draw_instance_predictions(predictions)
        # pi = cv2.cvtColor(v.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB)
        # axs[i].set_title(d["file_name"])
        pts = get_outer_vertex(seg[convex.vertices], im.shape[:2][1], im.shape[:2][0])

        sfile = save_masked_image(filepath, seg[convex.vertices])
        # 가로나 세로 양쪽이 너무 협소하지 않는지 검사
        pts_ok = check_pts(pts)
        if len(sfile) > 0 and pts_ok:
            from deskew import *

            image = cv2.imread(sfile)
            rotated = deskew_transform(image, pts, boxes[0])
            if rotated is not None:
                # axs[i + 1].imshow(cv2.cvtColor(rotated, cv2.COLOR_BGR2RGB))
                savefilename = "{}_box_transform.png".format(os.path.splitext(filepath)[0])
                cv2.imwrite(savefilename, rotated)
            rotated = deskew_transform(image, pts, None)
            if rotated is not None:
                # axs[i + 1].imshow(cv2.cvtColor(rotated, cv2.COLOR_BGR2RGB))
                savefilename = "{}_vs_transform.png".format(os.path.splitext(filepath)[0])
                cv2.imwrite(savefilename, rotated)

        i += 1

    # plt.show()
    # cv2.destroyAllWindows()